<h1 align="center">ECE 4438B Advanced Image Processing and Analysis</h1>
<h3 align="center">Assignment #4</h3>
<h4 align="center">Jianhui Li,  <a href="mailto:ljianhui@uwo.ca?subject=Assignment4">ljianhui@uwo.ca</a></h4>
<h4 align="center">Mechatronic Systems Engineering</h4>
<h4 align="center">Western University</h4>
<h4 align="center">Date of submission: April. 19th, 2019</h4>
<h4 align="center">Submitted to: Elvis C.S. Chen</h4>

**Question 1** ($5$ marks)

Assume the list of landmarks are recorded in two text files named `P.txt` and `V.txt`, each of the following formats:

```
x1 y1
x2 y2
...
xn yn
```

where the list of landmarks from image **A** is stored in `P.txt`, and landmarks from image **B** is stored in `V.txt`, respectively. A single white space " " is used to separate the values.

Consult with the `numpy.loadtxt` [documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html). Write a Python function that reads the text file and return them as a $2$D numpy array.

The function header should look like this:

``` python
def readLandmarksFromFile( fname ):
``` 
**(5 marks)** Correct implementation of this function. Because we will be using it to read 2 files, 
  * the file name(s) is the input of this function,
  * the output is a $2$D numpy array whose numerical value corresponds to the values stored in the text file,
 

In [3]:
import numpy as np
import SimpleITK as sitk

In [4]:
def readLandmarksFromFile( fname ):
    array = np.loadtxt(fname)
    return array

In [5]:
P = readLandmarksFromFile('P.txt') # upload matrix from .txt
V = readLandmarksFromFile('v.txt')
print(P)
print(V)

[[ 0.  1.]
 [-1.  0.]
 [ 0. -1.]
 [ 1.  0.]]
[[ 0.    0.75]
 [-1.    0.25]
 [ 0.   -1.25]
 [ 1.    0.25]]


**Question 2** ($10$ marks)

Implement the spatial function $U$ in $2$D for TPS for **a single homogous point-pair**. The function header should look like this:

``` python
def computeU( pi, vi ):
```

where

$
\begin{eqnarray}
p_i & = & \begin{bmatrix} x_i\\y_i\end{bmatrix}\\
v_i & = & \begin{bmatrix} x'_i\\y'_i\end{bmatrix}
\end{eqnarray}
$

are points in $P$ and $V$, respectively. The output of this function is a *scalar*.

**(10 marks)** Correct implementation of this function:
  * Using the class notes and Bookstein's paper, implement the correct formulation of $U$ for $2$D TPS.
  * complete and detailed documentation
  
*hints*
* using the notation on page 570 of Bookstein's paper, $r_{ij}$ is the Euclidean distance between points $i$ and point $j$. The function $U$ is a measure of the bending energy.


answer：

The least bent surface is given by the following equation:

$
T(x,y) = a_1 + a_2 x + a_3 y + \sum_{j=1}^n w_{j} U(| P_j - (x,y) |)
$

where $P_j$ denotes the $j^{th}$ landmark, $w_i$ is the weight of the basis function $U$ for $j^{th}$ landmark, and $| P_j-(x,y) |$ is the distance between the $j^{th}$ landmark (control point) and a position $(x,y)$.

$U$ is the a function based on the distance. In the $2$D case, 

$
\begin{eqnarray}
U(r) & = & r^2 \log(r^2)\\
r^2 & = & x^2 + y^2
\end{eqnarray}
$


$r_{i,j} = |(x_i,y_i)-(x_j,y_j)|$



In [6]:
def computeU( pi,vi):
    size = np.shape(pi)    # Get size of P
    n = size[0]            # how many points from P, the row number 
    r_2 = np.zeros((n,n))  # create r square array,, zeros(())
    for i in range (0,n):    
        for j in range (0,n):                                              #x^2+y^2 to calculate the distance of points in P
            r_2[i][j] = (pi[i][0]-pi[j][0])**2 + (pi[i][1]-pi[j][1])**2    # calculate r square for all points in P
                    
    U = np.zeros((n,n))    
    for i in range (0,n):    
        for j in range (0,n):
            if r_2[i][j] == 0:    # if r square = 0 copy to U to avoid log(0) error
                U[i][j] = 0
            else:
                U[i][j] = r_2[i][j]*np.log(r_2[i][j])    
                # calculate All U even though there are only 2 distance it is easier to construct the full array
    return U


In [7]:
U = computeU(P,V)
print(U)

[[0.         1.38629436 5.54517744 1.38629436]
 [1.38629436 0.         1.38629436 5.54517744]
 [5.54517744 1.38629436 0.         1.38629436]
 [1.38629436 5.54517744 1.38629436 0.        ]]


**Question 3** ($10$ marks)

Based on your implementation for Question 1 and 2, construct the matrix $K$ (page 570 on Bookstein's paper, or consult the class note). The function header should look like this:

``` python
def genK( P ):
```

where $P$ is the list of $2$D landmarks of image **A** that you read per Question 1. 

**(10 marks)** Correct implementation of this function
  * This function should determine the number of landmarks based on the size of $P$,
  * Uses the function `computeU` to fill in elements of the matrix $K$,
  * This functions returns a $2$D numpy array with correct dimension and values,
  * Complete and detailed documentation

$
\begin{eqnarray}
K & = &\begin{bmatrix} 0 & U(r_{1,2}) & \cdots & U(r_{1,n}) \\
U(r_{2,1}) & 0 & \cdots & U(r_{2,n}) \\
\cdots & \cdots & \cdots & \cdots \\
U(r_{n,1}) & U(r_{n,2}) & \cdots & 0 \\
\end{bmatrix}_{n \times n}\\
\end{eqnarray}
$

In [8]:
def genK( P ):
    # fill K matrix by U
    size = np.shape(P)    # Get size of P
    n = size[0]    # Get n -> how many points from P
    K = np.zeros((n,n))    # create K array
    U = computeU(P,V) 
    for i in range (0,n):
        for j in range (0,n):
            if i != j:     
                K[i][j] = U[i][j]   # fill the rest using U array
    return K

In [9]:
K = genK(P)
print(K)

[[0.         1.38629436 5.54517744 1.38629436]
 [1.38629436 0.         1.38629436 5.54517744]
 [5.54517744 1.38629436 0.         1.38629436]
 [1.38629436 5.54517744 1.38629436 0.        ]]


**Question 4** ($10$ marks)

Refer to page 570 on Bookstein's paper (or the class notes), construct the system of linear equations and solving them. Define the following $2$ functions:

``` python
def genL( P ):
``` 

and

``` python
def solveTPSCoefs( P, V )
```

where $P$ and $V$ are the landmarks you read from files (per Questions 1).

* `genL(P)` constructs the matrix $L$ which comprises of the matrix $K$ and the landmarks $P$.
* `solveTPSCoefs` computes the coefficients of the TPS transformation by multiplying the *inverse* of the matrix $L$ to the matrix $Y$, which itself is the landmarks $V$ with *zeros* appended.

$
L = \left[
\begin{array}{c|c}
K & P\\
\hline
P^{T} & 0
\end{array}
\right]
$



In [12]:
def genL(P):
    #create the P matrix with value 1 as the first element 
    size = np.shape(P)    # Get size of P
    rows = size[0]    # Get rows count from P
    cols = size[1]    # Get cols count from P
    N_P = np.ones((rows,cols+1))    # create new P
    for i in range (0,rows):
        for j in range (1,cols+1):
                N_P[i][j] = P[i][j-1]    # copy rest of P
    # create the L matrix            
    K = genK(P)
    Ksize = np.shape(K)    # Get size of K
    L_r = Ksize[0] + cols + 1    # determine rows number for L
    L_c = Ksize[1] + cols + 1    # determine col number for L
    L = np.zeros((L_r,L_c))    # create empty L array
    for i in range (0,Ksize[0]):
        for j in range (0,Ksize[1]):
            L[i][j] = K[i][j]    # fill K area
            
    for i in range (Ksize[0],L_r):
        for j in range (0,Ksize[1]):
            L[i][j] = np.transpose(N_P)[i-Ksize[0]][j]

    for i in range (0,Ksize[0]):
        for j in range (Ksize[1],L_c):
            L[i][j] = N_P[i][j-Ksize[1]]    
            
    
    return L

In [13]:
L = genL(P)
print(L)

[[ 0.          1.38629436  5.54517744  1.38629436  1.          0.
   1.        ]
 [ 1.38629436  0.          1.38629436  5.54517744  1.         -1.
   0.        ]
 [ 5.54517744  1.38629436  0.          1.38629436  1.          0.
  -1.        ]
 [ 1.38629436  5.54517744  1.38629436  0.          1.          1.
   0.        ]
 [ 1.          1.          1.          1.          0.          0.
   0.        ]
 [ 0.         -1.          0.          1.          0.          0.
   0.        ]
 [ 1.          0.         -1.          0.          0.          0.
   0.        ]]


#### Solve for $a$ and $b$

$ 
\left[ 
\begin{array}{c}
b \\
a\end{array}
\right] =  
\left[
\begin{array}{cc}
K & P\\
P^{T} & 0
\end{array}
\right]^{-1}  
\left[
\begin{array}{c}
V\\
0
\end{array}
\right]
$

In [14]:
def solveTPSCoefs(P,V):
    size = np.shape(V)    # Get size of V
    rows =size[0]    # Get rows count from V
    cols =size[1]    # Get cols count from V
    L = genL(P)
    L_size = np.shape(L)    # Get size of L
    c = L_size[1]    # Get cols count from L
    V_0 = np.zeros((c,cols))    # create V/0 array
    for i in range (0,rows):
        for j in range (0,cols):     # the upper 4 row, fill with V
            V_0[i][j] = V[i][j]    
    for i in range (rows,c):
        for j in range (0,cols):   # the bottom 3 row
            V_0[i][j] = 0    # fill zeros

    b_a = np.matmul(np.linalg.inv(L),V_0)   #  inverse L, and multiple V/0 array
    b_a = np.round((b_a), decimals=5)    # round to 5 decimal
    return b_a

In [15]:
Coef = solveTPSCoefs(P,V)  ## b / a: b 4 rows, a 3 rows
print(Coef)

[[ 0.      -0.09017]
 [ 0.       0.09017]
 [ 0.      -0.09017]
 [ 0.       0.09017]
 [ 0.      -0.     ]
 [ 1.      -0.     ]
 [ 0.       1.     ]]


## Validate your implementation

A sample `P.txt` and `V.txt` are provided, corresponding to Figure 2 of Bookstein's paper (and the class note). You can verify your implementation for Question 1-4 by referring to Page 571 of Bookstein's paper. 

**Question 5** ($20$ marks)

The TPS coefficients you just solved is encoded in a matrix of size $(n+3) \times 2$, where $n$ is the number of landmark pairs.. The first $(n+3) \times 1$ coefficients correspond to the deformation applied in the $x-$axis, and the second $(n+3) \times 1$ coefficients correspond to the deformation applined in the $y-$axis.  For a given pixel location $(x,y)$, implement a function that applies TPS deformation:

``` python
def applyTPSDeformation( P, V, pixelLocation )
```

where $P$ and $V$ are homologous landmarks, and

$
pixelLocation = \begin{bmatrix}x\\y\end{bmatrix}
$

is the input pixel location. The output of this function is

$
newPixelLocation = \begin{bmatrix}x'\\y'\end{bmatrix}
$

Suppose we have $4$ new pixels:

$
newPixels = \begin{bmatrix}100&0\\
-100&0\\
0&100\\
0&-100\end{bmatrix}
$

* ($2$ marks) Where will these $4$ pixels be transformed to?
* ($8$ marks) Have they been moved at all?  Is this what you expected?
* ($10$ marks) for implementation/code documentation.


## answer:
1) from the result, the four pixel will be transformed to [100 -0.25000433] [-100 0.25000433],[0 99.74999567][0 -99.74999567]

2) The points have been moved to location shown above. The movement is not exactly the same as P->V because they contains not only affine transformation, but also has Deformable Transformation (TPS), which has different effect on different points. 

3) the code is below


(in $2$D)
$
\begin{eqnarray}
x' & = & a_{1,1} + a_{2,1} x + a_{3,1} y + \sum_{j=1}^n w_{j,1} U(|(x_j,y_j)-(x,y)|)\\
y' & = & a_{1,2} + a_{2,2} x + a_{3,2} y + \sum_{j=1}^n w_{j,2} U(|(x_j,y_j)-(x,y)|)\\
\end{eqnarray}
$

In [16]:
def applyTPSDeformation( P, V, pixelLocation):
    Coef = solveTPSCoefs(P,V)
    x = pixelLocation[:,0]   # all x of the newpixel
    y = pixelLocation[:,1]   # all y of the newpixel
    n = Coef.shape[0] - 3     # get the length of w, 
    U = computeU(P,pixelLocation)
    xsum = 0
    ysum = 0
    #  Deformable Transformation (TPS) component
    #𝑤j is the weight of the basis function contributed by a landmark 𝑗
    for i in range(0,n):   # U[i]=U[i][j] = r_2[i][j]*np.log(r_2[i][j]) 
        xsum += Coef[i][0]*U[i]    # calculate accumulated w*U for x, the first four
        ysum += Coef[i][1]*U[i]    # calculate accumulated w*U for y
        
    #  Affine Transformation component a_i + a_2 x + a_3 y
    n_x = Coef[n][0]+Coef[n+1][0]*x+Coef[n+2][0]*y + xsum    # calculate new x the last three
    n_y = Coef[n][1]+Coef[n+1][1]*x+Coef[n+2][1]*y + ysum    # calculate new y
    n_xy = np.array([n_x,n_y])
    
    return n_xy

In [17]:
pixelLocation = np.array([[100, 0],[-100, 0],[0, 100],[0, -100]])
new_pixel = applyTPSDeformation(P,V,pixelLocation)
print(new_pixel[:,0])
print(new_pixel[:,1])
print(new_pixel[:,2])
print(new_pixel[:,3])

[100.          -0.25000433]
[-100.            0.25000433]
[ 0.         99.74999567]
[  0.         -99.74999567]


**Question 6** ($20$ marks)

Create $2$ text files corresponding to the landmarks as shown in Figure 4 of Bookstein's paper (numerical values are given on page 572). Use your solution for Question 4 to generate TPS coefficients and verify that you are getting the same answer to Bookstein's example.

(**hint**: One of the coefficients was mis-typed in the paper so don't be too concerned if the numbers don't match up exactly)

Create a matrix $A_{2\times 2}$ corresponding to the linear term of the affine transformation coefficients. Perform [Singular Value Decomposition](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.svd.html) on $A$ ($A = U S V^{H}$) decomposes the affine transformation into a rotation, followed by scaling, followed by another rotation. The *singular values* $S$ correspond to the amount of scaling, and $U$ and $V^{H}$ are rotations.

**hint**: If the determine of $U$ and $V^{H}$ is $-1$, a flip is performed twice!  i.e. SVD solution is **not** unique. To obtain proper rotation, apply the following (un)-flip:

$
\begin{eqnarray}
U & = & \begin{bmatrix}-1 & 0 \\0&1\end{bmatrix} U\\
V^{H} & = & \begin{bmatrix}-1 & 0 \\0&1\end{bmatrix} V^{H}
\end{eqnarray}
$

(**Marks**)
* ($5$ marks) What is the amount of rotations, in degrees, does $U$ and $V^{H}$ represent?
* ($5$ marks) What are the anisotropic scaling factors?
* ($10$ marks) In your own words, describe what the affine transformation $A$ does as part of the TPS transformation. In your discussion, elaborate how landmarks are moved/transformed.


## answer:
1) Degree of U: -53.33; Degree of VH: -44.89 

2) Scaling factor: 1.0719, 0.7441

3) From the equation, the first matrix is VH, then S, then U. The affine transformation A is in a direction 44.89 degree clockwise, and extension by a factor 1.0719 in x direction compression by 0.7441 in the y direction of the original coordinate, then followed by a rotation of another 8.45 degree clockwise.

In [18]:
P2 = readLandmarksFromFile('Pnew.txt')
V2 = readLandmarksFromFile('Vnew.txt')
Coef2 = solveTPSCoefs(P2,V2)
print(Coef2)

[[-0.03803  0.04244]
 [ 0.02319  0.01592]
 [-0.02476  0.02881]
 [ 0.07978 -0.04542]
 [-0.04018 -0.04175]
 [ 1.355   -2.94616]
 [ 0.87473 -0.29553]
 [-0.02886  0.92164]]


In [19]:
#linear term of the affine transformation coefficients
# find the x, and y for matrix A
A = np.array([[Coef2[6][0], Coef2[7][0]], [Coef2[6][1], Coef2[7][1]]])
U, S, VH = np.linalg.svd(A, full_matrices=True)    # perform singular value decomposition
print('A',A)
print('U:',U)
print('S:',S) # scaling factor 
print('VH:',VH)
np.linalg.det(U) # determine is -1, therefore, need to multiple -1
np.linalg.det(VH) 

A [[ 0.87473 -0.02886]
 [-0.29553  0.92164]]
U: [[-0.5971188   0.80215281]
 [ 0.80215281  0.5971188 ]]
S: [1.07190995 0.74414568]
VH: [[-0.70843446  0.7057766 ]
 [ 0.7057766   0.70843446]]


-1.0

In [20]:
T = np.array([[-1,0],[0,1]])
U2 = np.matmul(T, U)    # flip U back, mutiple
VH2 = np.matmul(T, VH)    # flip VH back, multiple
print('U:',U2)
print('VH:',VH2)
Urad=np.arcsin(U2[0,1])    #Find the U rotation in rads 
Udeg=Urad*180/np.pi     #Find U rotation in degrees
print(Udeg)
VHrad=np.arcsin(VH2[0,1]) #find the VH rotation in rads 
VHdeg=VHrad*180/np.pi   #Find the VH rotation in degrees 
print(VHdeg)
D2=Udeg-VHdeg    #second rotation
print(D2)

U: [[ 0.5971188  -0.80215281]
 [ 0.80215281  0.5971188 ]]
VH: [[ 0.70843446 -0.7057766 ]
 [ 0.7057766   0.70843446]]
-53.33617538649806
-44.892318856951405
-8.443856529546657
